In [4]:
from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import os 

os.getcwd()

os.listdir('/content/drive/MyDrive/Deeplabv3Flat')
os.listdir('/content/drive/MyDrive/Datasets')

os.chdir('/content/drive/MyDrive/Deeplabv3Flat')
print('Directory Changed')

Directory Changed


In [ ]:
# camera-ready

import sys
! pip install import-ipynb
import import_ipynb

sys.path.append("/content/drive/MyDrive/Deeplabv3Flat")
from datasets import DatasetVal # (this needs to be imported before torch, because cv2 needs to be imported before torch for some reason)

# No need to append with the flat file structure
#sys.path.append("/root/deeplabv3/model")
from deeplabv3 import DeepLabV3

#sys.path.append("/content/drive/MyDrive/Deeplabv3Flat")
from utils import label_img_to_color

import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

batch_size = 2

best_trained_model = 'model_2_epoch_85.pth' # NB put the file name of best weight checkpoint 

network = DeepLabV3("eval_val", project_dir="/content/drive/MyDrive/Deeplabv3Flat").cuda()
network.load_state_dict(torch.load("/content/drive/MyDrive/Deeplabv3Flat/training_logs/model_2/checkpoints/" + best_trained_model))

val_dataset = DatasetVal(cityscapes_data_path="/content/drive/MyDrive/Datasets/Cityscapes",
                         cityscapes_meta_path="/content/drive/MyDrive/Datasets/Cityscapes/meta")

num_val_batches = int(len(val_dataset)/batch_size)
print ("num_val_batches:", num_val_batches)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size, shuffle=False,
                                         num_workers=1)

with open("/content/drive/MyDrive/Datasets/Cityscapes/meta/class_weights.pkl", "rb") as file: # (needed for python3)
    class_weights = np.array(pickle.load(file))
class_weights = torch.from_numpy(class_weights)
class_weights = Variable(class_weights.type(torch.FloatTensor)).cuda()

# loss function
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

network.eval() # (set in evaluation mode, this affects BatchNorm and dropout)
batch_losses = []
for step, (imgs, label_imgs, img_ids) in enumerate(val_loader):
    with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        label_imgs = Variable(label_imgs.type(torch.LongTensor)).cuda() # (shape: (batch_size, img_h, img_w))

        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))  # imgs are the raw images from the cityscapes validation folder 

        # compute the loss:
        loss = loss_fn(outputs, label_imgs)
        loss_value = loss.data.cpu().numpy()
        batch_losses.append(loss_value)

        ########################################################################
        # save data for visualization:
        ########################################################################
        outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
        pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
        pred_label_imgs = pred_label_imgs.astype(np.uint8)

        for i in range(pred_label_imgs.shape[0]):
            if i == 0:
                pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
                img_id = img_ids[i]
                img = imgs[i] # (shape: (3, img_h, img_w))

                img = img.data.cpu().numpy()
                img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
                img = img*np.array([0.229, 0.224, 0.225])
                img = img + np.array([0.485, 0.456, 0.406])
                img = img*255.0
                img = img.astype(np.uint8)

                pred_label_img_color = label_img_to_color(pred_label_img)
                overlayed_img = 0.35*img + 0.65*pred_label_img_color
                overlayed_img = overlayed_img.astype(np.uint8)

                cv2.imwrite(network.model_dir + "/" + img_id + "_overlayed.png", overlayed_img)

val_loss = np.mean(batch_losses)
print ("val loss: %g" % val_loss)


importing Jupyter notebook from datasets.ipynb
importing Jupyter notebook from deeplabv3.ipynb
importing Jupyter notebook from resnet.ipynb
importing Jupyter notebook from aspp.ipynb
importing Jupyter notebook from utils.ipynb
pretrained resnet, 18
num_val_batches: 250


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3328: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3458: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


val loss: 0.538642


In [15]:
# visualising the matrix of the output trying to bug test for the colour mismatch from the gt labels. 

import sys
! pip install import-ipynb
import import_ipynb

sys.path.append("/content/drive/MyDrive/Deeplabv3Flat")
from datasets import DatasetVal # (this needs to be imported before torch, because cv2 needs to be imported before torch for some reason)

# No need to append with the flat file structure
#sys.path.append("/root/deeplabv3/model")
from deeplabv3 import DeepLabV3

#sys.path.append("/content/drive/MyDrive/Deeplabv3Flat")
from utils import label_img_to_color

import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F

import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

batch_size = 2

best_trained_model = 'model_2_epoch_85.pth' # NB put the file name of best weight checkpoint 

network = DeepLabV3("eval_val", project_dir="/content/drive/MyDrive/Deeplabv3Flat").cuda()
network.load_state_dict(torch.load("/content/drive/MyDrive/Deeplabv3Flat/training_logs/model_2/checkpoints/" + best_trained_model))

val_dataset = DatasetVal(cityscapes_data_path="/content/drive/MyDrive/Datasets/Cityscapes",
                         cityscapes_meta_path="/content/drive/MyDrive/Datasets/Cityscapes/meta")

num_val_batches = int(len(val_dataset)/batch_size)
print ("num_val_batches:", num_val_batches)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=batch_size, shuffle=False,
                                         num_workers=1)

with open("/content/drive/MyDrive/Datasets/Cityscapes/meta/class_weights.pkl", "rb") as file: # (needed for python3)
    class_weights = np.array(pickle.load(file))
class_weights = torch.from_numpy(class_weights)
class_weights = Variable(class_weights.type(torch.FloatTensor)).cuda()

# loss function
loss_fn = nn.CrossEntropyLoss(weight=class_weights)

trainId_to_id = {
    0: 7,
    1: 8,
    2: 11,
    3: 12,
    4: 13,
    5: 17,
    6: 19,
    7: 20,
    8: 21,
    9: 22,
    10: 23,
    11: 24,
    12: 25,
    13: 26,
    14: 27,
    15: 28,
    16: 31,
    17: 32,
    18: 33,
    19: 0
}
trainId_to_id_map_func = np.vectorize(trainId_to_id.get)

def label_img_to_color_BGR(img):
    label_to_color = {
        0:  [128, 64,128],
        1:  [232, 35,244],
        2:  [ 70, 70, 70],
        3:  [156,102,102],
        4:  [153,153,190],
        5:  [153,153,153],
        6:  [30 ,170,250],
        7:  [  0,220,220],
        8:  [35 ,142,107],
        9:  [152,251,152],
        10: [180,130, 70],
        11: [ 60, 20,220],
        12: [  0,  0,255],
        13: [142,  0,  0],
        14: [ 70,  0,  0],
        15: [100, 60,  0],
        16: [100, 80,  0],
        17: [230,  0,  0],
        18: [ 32, 11,119],
        19: [81,  0,  81]
        }

    img_height, img_width = img.shape

    img_color = np.zeros((img_height, img_width, 3))
    for row in range(img_height):
        for col in range(img_width):
            label = img[row, col] #this cycles through the whole array of predicted values and grabs the label at that point

            img_color[row, col] = np.array(label_to_color[label]) # this slots in the colour vector for the specific label from the list above 

    return img_color


network.eval() # (set in evaluation mode, this affects BatchNorm and dropout)
batch_losses = []
for step, (imgs, label_imgs, img_ids) in enumerate(val_loader):
    with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        label_imgs = Variable(label_imgs.type(torch.LongTensor)).cuda() # (shape: (batch_size, img_h, img_w))
        print("________________________")
        print("LABEL_IMGS.SHAPE", label_imgs.shape)


        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))  # imgs are the raw images from the cityscapes validation folder 
        print("________________________")
        print("OUTPUTS.SHAPE", outputs.shape)

        # compute the loss:
        loss = loss_fn(outputs, label_imgs)  # this works out cross entropy of predicted labels with gt labels (uses the weights too)
        loss_value = loss.data.cpu().numpy()
        batch_losses.append(loss_value)

        ########################################################################
        # save data for visualization:
        ########################################################################
        outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
        print("________________________")
        print("OUTPUTS:", outputs)
        
        pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
        pred_label_imgs = pred_label_imgs.astype(np.uint8)
        
        print("PRED_LABEL_IMGS:", pred_label_imgs)
        
        print(pred_label_imgs[1])
        
        print("________________________")
        print("PRED_LABEL_IMGS.SHAPE", pred_label_imgs.shape)
        print("________________________")
        print("pred_label_imgs.shape[0]", pred_label_imgs.shape[0]) # this gives the batch size. 2 in this case 
        print("________________________")
        print("img_ids[0]", img_ids[0])
        print("________________________")
        print("pred_label_imgs[0]", pred_label_imgs[0])
        print("________________________")
        print("img_ids", img_ids)

        for i in range(pred_label_imgs.shape[0]): # gives batch size
            if i == 0: 
            # this means we only creates the overlayed image for the first picture in the batch. Since the batch size is 2 that means we only create overlaid images for half of the val images.. probs to save time/space
            # notice eval_on_val_for_metrics doesnt do this as we need each image for the metrics. 
                pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
                img_id = img_ids[i]
                img = imgs[i] # (shape: (3, img_h, img_w))

                img = img.data.cpu().numpy()
                img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
                img = img*np.array([0.229, 0.224, 0.225])
                img = img + np.array([0.485, 0.456, 0.406])
                img = img*255.0
                img = img.astype(np.uint8)

                
                # convert pred_label_img from trainId to id pixel values: I added this from the eval_on_val_for_metrics script. 
                #pred_label_img = trainId_to_id_map_func(pred_label_img) # (shape: (1024, 2048))
                #pred_label_img = pred_label_img.astype(np.uint8)

                pred_label_img_color = label_img_to_color_BGR(pred_label_img) 
                print("*****")
                print(img)
                
                print(img_id)
                print("____________")
                print(pred_label_img)
                print("____________")
                print(pred_label_img_color)
                # pred_label_img_color_bgr = cv2.cvtColor(pred_label_img_color, cv2.COLOR_BGR2RGB) # becuase we save the image with opencv below and it has the convention BGR 

                overlayed_img = 0.35*img + 0.65*pred_label_img_color
                overlayed_img = overlayed_img.astype(np.uint8)

                print("____________")
                print("Network.model_dir", network.model_dir)

                
                cv2.imwrite(network.model_dir + "/" + img_id + "_colour_test.png", pred_label_img_color)
                cv2.imwrite(network.model_dir + "/" + img_id + "_overlayed.png", overlayed_img)
            break
        
    break


val_loss = np.mean(batch_losses)
print ("val loss: %g" % val_loss)


pretrained resnet, 18
num_val_batches: 250
________________________
LABEL_IMGS.SHAPE torch.Size([2, 512, 1024])
________________________
OUTPUTS.SHAPE torch.Size([2, 20, 512, 1024])
________________________
OUTPUTS: [[[[-16.40648    -16.40648    -16.40648    ... -11.184682
    -11.184682   -11.184682  ]
   [-16.40648    -16.40648    -16.40648    ... -11.184682
    -11.184682   -11.184682  ]
   [-16.40648    -16.40648    -16.40648    ... -11.184682
    -11.184682   -11.184682  ]
   ...
   [ -9.045122    -9.045122    -9.045122   ...  -1.3342888
     -1.3342888   -1.3342888 ]
   [ -9.045122    -9.045122    -9.045122   ...  -1.3342888
     -1.3342888   -1.3342888 ]
   [ -9.045122    -9.045122    -9.045122   ...  -1.3342888
     -1.3342888   -1.3342888 ]]

  [[-10.0230875  -10.0230875  -10.0230875  ...  -8.861837
     -8.861837    -8.861837  ]
   [-10.0230875  -10.0230875  -10.0230875  ...  -8.861837
     -8.861837    -8.861837  ]
   [-10.0230875  -10.0230875  -10.0230875  ...  -8.861837
  

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3487: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3613: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


PRED_LABEL_IMGS: [[[ 8  8  8 ... 19 19 19]
  [ 8  8  8 ... 19 19 19]
  [ 8  8  8 ... 19 19 19]
  ...
  [19 19 19 ... 19 19 19]
  [19 19 19 ... 19 19 19]
  [19 19 19 ... 19 19 19]]

 [[ 7  7  7 ... 19 19 19]
  [ 7  7  7 ... 19 19 19]
  [ 7  7  7 ... 19 19 19]
  ...
  [19 19 19 ... 19 19 19]
  [19 19 19 ... 19 19 19]
  [19 19 19 ... 19 19 19]]]
[[ 7  7  7 ... 19 19 19]
 [ 7  7  7 ... 19 19 19]
 [ 7  7  7 ... 19 19 19]
 ...
 [19 19 19 ... 19 19 19]
 [19 19 19 ... 19 19 19]
 [19 19 19 ... 19 19 19]]
________________________
PRED_LABEL_IMGS.SHAPE (2, 512, 1024)
________________________
pred_label_imgs.shape[0] 2
________________________
img_ids[0] frankfurt_000001_048196
________________________
pred_label_imgs[0] [[ 8  8  8 ... 19 19 19]
 [ 8  8  8 ... 19 19 19]
 [ 8  8  8 ... 19 19 19]
 ...
 [19 19 19 ... 19 19 19]
 [19 19 19 ... 19 19 19]
 [19 19 19 ... 19 19 19]]
________________________
img_ids ('frankfurt_000001_048196', 'frankfurt_000001_055062')
*****
[[[  2  21   3]
  [  6  24   7]